In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
import itertools

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

from scipy import stats
from scipy.stats import pearsonr
import pingouin as pg
import os
import importlib
import utilities
%matplotlib inline

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.11, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


# load semspace

In [2]:
importlib.reload(utilities)

<module 'utilities' from '/Users/yyu/PycharmProjects/AUT-Scoring/scripts/utilities.py'>

In [8]:
splist=['cbow_6_ukwac_subtitle','cbow_subtitle','glove_6B','TASA','ukwac']
folder='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/'
semspace_dict = {}

for sp in splist:
    if sp=='ukwac':
        semspace_dict[sp] = pd.read_csv(folder+'/semantic_spaces/'+sp+'.txt', delimiter = " ", header = None,encoding = "ISO-8859-1").set_index(0)
    else:
        semspace_dict[sp] = pd.read_csv(folder+'/semantic_spaces/'+sp+'.txt', delimiter = " ", header = None).set_index(0)


# SemDis Studies

In [24]:
studytag='s3'
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_data_long.xlsx'
df=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_long')
df.head()

,id,item,response,time,br_rater1,br_rater2,br_rater3,br_mean,cbowu,cbows,cboww,tasa,glov
0,1,brick,necklace,0.17,2.0,1,2,1.67,0.82610,0.91289,0.88754,1.08987,0.90976
1,1,brick,shoes,0.29,1.0,1,2,1.33,0.72027,0.96824,0.84282,1.10507,0.82479
2,1,brick,car,0.44,2.0,1,1,1.33,0.73301,0.87752,0.88443,0.95250,0.82105
3,1,brick,hat,0.64,1.0,1,2,1.33,0.72628,0.85444,0.72550,1.03804,0.88256
4,1,brick,plate,0.81,2.0,2,2,2.00,0.84976,0.81647,0.86750,0.99488,0.83501


In [25]:

#lemmatize
df['response_processed']=df.apply(lambda r: utilities.lemmatize(r['response'],utilities.stopwords_edited), axis=1)
#remove stim word
df['response_processed']=df.apply(lambda r: [x for x in r['response_processed'] if x!=r['item']], axis=1)


In [5]:
#for s1
yy_metrics=['ewm_'+sp for sp in splist]+['mad_'+sp for sp in splist]+['wad_'+sp for sp in splist]
sd_metrics=['cboww_m','cbowu_m','cbows_m', 'tasa_m', 'glov_m']
rater_metrics=['rater1','rater2','rater3','rater4']
df['rater']=df[rater_metrics].mean(axis=1)

In [17]:
#for s2
yy_metrics=['ewm_'+sp for sp in splist]+['mad_'+sp for sp in splist]+['wad_'+sp for sp in splist]
sd_metrics=['cboww','cbows','cbowu', 'tasa', 'glov']
rater_metrics=['rater1','rater2','rater3']
df['rater']=df[rater_metrics].mean(axis=1)

In [27]:
#for s3
yy_metrics=['ewm_'+sp for sp in splist]+['mad_'+sp for sp in splist]+['wad_'+sp for sp in splist]
sd_metrics=['cboww','cbows','cbowu', 'tasa', 'glov']
df['rater']=df['br_mean']

In [41]:
#for s5
yy_metrics=['ewm_'+sp for sp in splist]+['min_'+sp for sp in splist]+['orig_'+sp for sp in splist]
sd_metrics=['cboww','cbows','cbowu', 'tasa', 'glov']
rater_metrics=['rater1','rater2','rater3']
df['rater']=df[rater_metrics].mean(axis=1)

In [24]:
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
agg.head()

,id,age,gender,br_rater1,br_rater2,br_rater3,br_mean,br_cbowu,br_cbows,br_cboww,...,e_ffi,e_ffi.1,a_ffi,c_ffi,meta1_r1,meta1_r2,meta1_r3,meta2_r1,meta2_r2,meta2_r3
0,1,25,m,1.38,1.23,1.62,1.41,0.80,0.90,0.86,...,3.33,3.00,4.08,4.92,1,1,1,1,1,1
1,2,18,m,1.25,1.25,1.75,1.42,0.99,0.98,0.99,...,2.75,2.83,3.25,3.25,2,1,4,1,1,1
2,3,20,m,1.40,1.20,2.40,1.67,0.95,0.99,0.97,...,3.92,3.58,3.83,4.00,3,3,3,3,3,2
3,4,18,f,2.00,1.46,2.92,2.13,1.01,1.00,1.00,...,3.42,3.08,3.92,3.92,1,1,2,1,1,3
4,5,18,f,1.50,1.67,2.83,2.00,1.00,0.95,0.99,...,4.00,3.83,3.83,3.83,2,1,2,3,2,1


## novelty

In [28]:
for sp in splist:
    df['wad'+sp]=df.apply(lambda r: utilities.get_weightedscore(r['item'], r['response_processed'],semspace_dict[sp]), axis=1)
    df['ewm_'+sp]=df.apply(lambda r: utilities.get_ewm_cosdist(r['item'], r['response_processed'], semspace_dict[sp]), axis=1)
    df['mad_'+sp]=df.apply(lambda r: utilities.get_minvec(r['item'], r['response_processed'], semspace_dict[sp]), axis=1)
  


cannot find pully
cannot find lockedyou
cannot find theyou
cannot find openit
cannot find 4
cannot find illusional
cannot find exfoliant
cannot find withlike
cannot find flattener
cannot find bookbags
cannot find oldowan
cannot find brickball
cannot find brickly
cannot find potholder
cannot find tetri
cannot find 2
cannot find smoosh
cannot find incents
cannot find 3
cannot find 3
cannot find exfoliator
cannot find 2
cannot find tenderizing
cannot find preschooler
cannot find bookbags
cannot find unlevel
cannot find classroomits
cannot find lac3
cannot find terrac3
cannot find flowerscould
cannot find bookbag
cannot find structuresfor
cannot find artto
cannot find stoolvery
cannot find drinksso
cannot find terabithia
cannot find sculp
cannot find 6
cannot find wallif
cannot find exfoliant
cannot find doorjamb
cannot find cornhole
cannot find cornhole
cannot find tenderizer
cannot find tenderizer
cannot find tenderize
cannot find immobilizing
cannot find tetri
cannot find lonelywith
can

cannot find brickball
cannot find brickly
cannot find exfoliate
cannot find youll
cannot find tetri
cannot find myofascial
cannot find youre
cannot find smoosh
cannot find incents
cannot find handler
cannot find exfoliator
cannot find diy
cannot find jenga
cannot find bookbags
cannot find casted
cannot find footrest
cannot find bookend
cannot find schoolers
cannot find theyre
cannot find classroomits
cannot find withstands
cannot find bbq
cannot find lac3
cannot find terrac3
cannot find flowerscould
cannot find bookbag
cannot find structuresfor
cannot find artto
cannot find scrapper
cannot find bludgeon
cannot find stoolvery
cannot find drinksso
cannot find terabithia
cannot find bookend
cannot find guestbook
cannot find sculp
cannot find playset
cannot find wallif
cannot find exfoliant
cannot find bookend
cannot find cornhole
cannot find cornhole
cannot find flowerbed
cannot find youre
cannot find anorexic
cannot find jammer
cannot find bookend
cannot find microgravity
cannot find kar

cannot find withlike
cannot find firepit
cannot find colored
cannot find pouring
cannot find scratched
cannot find flattener
cannot find bookbags
cannot find oldowan
cannot find swatter
cannot find brickball
cannot find brickly
cannot find loofah
cannot find sculpting
cannot find potholder
cannot find tarp
cannot find tetri
cannot find myofascial
cannot find trying
cannot find taller
cannot find putting
cannot find 2
cannot find destroying
cannot find taller
cannot find parked
cannot find propter
cannot find stacked
cannot find tossing
cannot find backboard
cannot find smoosh
cannot find incents
cannot find 3
cannot find 3
cannot find scratching
cannot find exfoliator
cannot find anchoring
cannot find parked
cannot find 2
cannot find weighing
cannot find nailing
cannot find parked
cannot find scratcher
cannot find jell
cannot find tenderizing
cannot find jenga
cannot find preschooler
cannot find bookbags
cannot find unlevel
cannot find casted
cannot find sawed
cannot find sawed
cannot 

In [29]:
df.columns

Index(['id', 'item', 'response', 'time', 'br_rater1', 'br_rater2', 'br_rater3',
       'br_mean', 'cbowu', 'cbows', 'cboww', 'tasa', 'glov',
       'response_processed', 'rater', 'wadcbow_6_ukwac_subtitle',
       'ewm_cbow_6_ukwac_subtitle', 'mad_cbow_6_ukwac_subtitle',
       'wadcbow_subtitle', 'ewm_cbow_subtitle', 'mad_cbow_subtitle',
       'wadglove_6B', 'ewm_glove_6B', 'mad_glove_6B', 'wadTASA', 'ewm_TASA',
       'mad_TASA', 'wadukwac', 'ewm_ukwac', 'mad_ukwac'],
      dtype='object')

In [30]:
df.rename(columns=dict([('ewm_'+sp,'ewm_'+utilities.spNameMap[sp]) for sp in splist]+
                         [('mad_'+sp,'mad_'+utilities.spNameMap[sp]) for sp in splist]+
                         [('wad'+sp,'wad_'+utilities.spNameMap[sp]) for sp in splist]),inplace=True)

In [31]:
np.round(df,3).to_csv(sourceFolder+studytag+'_yy_long.csv',index=False)

## originality


In [119]:
for stim in df['item'].unique():
    responses=df[df['item']==stim]['response_processed'].values.tolist()
    for sp in splist:
        print(stim,sp)
        clusters=utilities.cluster_count_vectors(responses, True,semspace_dict[sp],repeat=5,max_cluster=250)
        clusterDF=clusters[0]
        cnt=np.bincount(clusterDF[clusterDF['category']>=0]['category'].values)
        clusterDF['freq']=clusterDF.apply(lambda r: cnt[r['category']] if r['category']>=0 else np.nan, axis=1)
        df.loc[df['item']==stim,'originality_'+sp]=(1-clusterDF['freq']/(clusterDF['freq'].sum())).values

bureau cbow_6_ukwac_subtitle
cannot find crat
30 0.34128791433910166 [3.  3.9 4. ]
40 0.36529970250923716 [2. 2. 3.]
50 0.41064408232670435 [1. 1. 2.]
# of clusters for 232 : 50
    category                                          responses
0         -1                                           [[crat]]
1          0  [[cabinet], [cabinet], [cabinet], [cabinet], [...
2          1  [[agency], [agency], [agency], [agency], [agen...
3          4  [[money], [bank], [finance], [glacial], [branch]]
4          5                            [[bookcase], [armoire]]
5          6  [[sophisticated], [system], [subset], [hierarc...
6          7                                           [[cake]]
7          8  [[government], [government], [government], [go...
8         10  [[division], [department], [department], [depa...
9         11  [[group], [group], [group], [club], [group], [...
10        12  [[tax], [corrupt], [education], [politician], ...
11        13                                          

30 0.3608600984266907 [4. 4. 5.]
40 0.40579776270475637 [2.  2.9 3.8]
50 0.4291950199720925 [1.45 2.   2.  ]
60 0.4605223790797379 [1. 1. 2.]
# of clusters for 232 : 60
    category                                          responses
0         -1                                           [[crat]]
1          0  [[group], [group], [group], [group], [group], ...
2          1                                         [[system]]
3          2                                           [[bank]]
4          3                                   [[help], [draw]]
5          4  [[firm], [firm], [business], [depression], [fi...
6          5                        [[tedious], [work], [busy]]
7          6        [[service], [postal], [service], [service]]
8          7  [[division], [corporation], [order], [division...
9          8                                         [[active]]
10         9                   [[proper], [center], [oklahoma]]
11        10                                         [[sturdy]]

30 0.4019397938496998 [2.  2.9 4. ]
40 0.4625274674136619 [2. 2. 2.]
50 0.4914012542070244 [1. 1. 2.]
# of clusters for 232 : 50
    category                                          responses
0         -1                                               [[]]
1          0  [[commerce], [commerce], [commerce], [commerce...
2          1  [[room], [room], [room], [room], [room], [room...
3          5      [[judge], [judge], [judge], [judge], [judge]]
4          6                                          [[suite]]
5          7            [[death], [confined], [jail], [prison]]
6          8                                        [[curtain]]
7          9  [[secret], [secret], [secret], [secret], [secr...
8         13                             [[opening], [section]]
9         14  [[network], [cell], [cell], [cell], [domain], ...
10        15                                          [[choir]]
11        16                                  [[bowl], [spoon]]
12        17                           

cannot find thanatology
30 0.309661840201037 [3.45 4.   5.  ]
40 0.3625161983572821 [2. 2. 3.]
50 0.40885795568787353 [2. 2. 2.]
60 0.4509145062604346 [1. 2. 2.]
70 0.46104440646923434 [1. 1. 1.]
# of clusters for 232 : 70
    category                                          responses
0         -1            [[], [], [], [], [thanatology], [], []]
1          0                                    [[note], [pen]]
2          1                                        [[suicide]]
3          2                                        [[certain]]
4          3                              [[sad], [sad], [sad]]
5          4                                          [[dying]]
6          5  [[black], [pale], [black], [hole], [black], [b...
7          6                               [[tombstone], [old]]
8          7   [[life], [life], [life], [life], [life], [life]]
9         10                                            [[pas]]
10        11                          [[easy], [stiff], [slow]]
11       

    category                                          responses
0         -1             [[decapitate], [], [], [], [], [], []]
1          0                                          [[water]]
2          1                                         [[island]]
3          2  [[black], [separated], [black], [black], [black]]
4          3                                     [[old], [age]]
5          4                    [[unknown], [great], [unknown]]
6          5   [[illness], [loss], [sickle], [loss], [illness]]
7          6                                        [[chapter]]
8          7                                        [[extinct]]
9          8  [[end], [end], [end], [end], [end], [end], [en...
10        12                                 [[decay], [teeth]]
11        13                           [[rest], [slow], [body]]
12        14                                          [[avoid]]
13        15  [[final], [final], [final], [final], [final], ...
14        17       [[forever], [forever]

...

cannot find welleducated
30 0.4437308032934531 [2.45 3.   3.8 ]
40 0.49426048408902074 [1. 2. 2.]
50 0.5335835246133264 [1.  1.  1.8]
# of clusters for 232 : 50
    category                                          responses
0         -1                               [[], [welleducated]]
1          0                         [[authority], [authority]]
2          1                             [[trainer], [trainer]]
3          2                                       [[creative]]
4          3          [[advice], [opinion], [advice], [advice]]
5          4                                           [[fast]]
6          5  [[specialist], [specialist], [specialist], [sp...
7          6                        [[winner], [ace], [winner]]
8          7             [[effective], [superior], [competent]]
9          8   [[best], [best], [best], [best], [best], [best]]
10        10                      [[dummy], [yellow], [monkey]]
11        11                   [[degree], [academic], [tenure]]
12     

30 0.3840329844636234 [1.45 2.   4.  ]
40 0.43576674338942706 [2.  2.  2.8]
50 0.464252240676252 [1. 1. 2.]
# of clusters for 232 : 50
    category                                          responses
0         -1                                       [[], [], []]
1          0                                            [[dog]]
2          1  [[sweatshirt], [sweatshirt], [sweatshirt], [sw...
3          2  [[car], [car], [car], [car], [car], [car], [ca...
4          3         [[money], [robbery], [robbery], [shooter]]
5          4   [[neighbor], [neighbor], [neighbor], [neighbor]]
6          7                    [[vehicle], [suspect], [model]]
7          8  [[rain], [sunny], [rain], [rain], [rain], [hate]]
8          9  [[visor], [clothing], [sweater], [dairy], [low...
9         10                                     [[lid], [cup]]
10        11                   [[head], [head], [head], [head]]
11        12                                            [[rag]]
12        13                     

30 0.45676570908665426 [2. 2. 3.]
40 0.5226990013317869 [1. 2. 2.]
# of clusters for 232 : 40
    category                                          responses
0          0                                        [[mandate]]
1          1                                           [[rain]]
2          2                                     [[parliament]]
3          3     [[statute], [statute], [tort], [constitution]]
4          4                           [[iced], [tea], [maker]]
5          5                         [[letter], [book], [book]]
6          6                             [[command], [command]]
7          7  [[confusing], [suggestion], [structure], [cons...
8          8                         [[crime], [crime], [perp]]
9          9  [[common], [abiding], [abiding], [degree], [co...
10        10  [[power], [control], [control], [land], [land]...
11        11  [[justice], [justice], [justice], [justice], [...
12        12  [[abide], [practice], [dictator], [rulebook], ...
13        

cannot find smackaroos
cannot find nontransparent
cannot find skrilla
30 0.40974690687797716 [2.  2.9 3. ]
40 0.4633302821668708 [1.  2.  2.8]
50 0.4709959215305149 [1. 1. 2.]
# of clusters for 232 : 50
    category                                          responses
0         -1  [[moola], [moola], [moola], [smackaroos], [non...
1          0                                   [[debt], [debt]]
2          1                                       [[resource]]
3          2                             [[finance], [finance]]
4          3                   [[treasure], [gold], [treasure]]
5          4                              [[financial], [bank]]
6          5                   [[status], [status], [security]]
7          6  [[cash], [cash], [cash], [cash], [cash], [cash...
8          9  [[wealth], [wealth], [wealth], [wealth], [weal...
9         13  [[salary], [payment], [pay], [purchase], [rent...
10        14  [[currency], [currency], [currency], [currency...
11        15                 

cannot find greenies
cannot find smackaroos
cannot find moolah
cannot find moolah
cannot find moolah
cannot find nontransparent
cannot find skrilla
cannot find lucre
30 0.43763930423870184 [2.45 3.   3.  ]
40 0.4804429601169916 [1.95 2.   2.  ]
50 0.49369795114515846 [1. 1. 2.]
# of clusters for 232 : 50
    category                                          responses
0         -1  [[moola], [moola], [moola], [dinero], [moolah]...
1          0                                         [[change]]
2          1  [[rich], [rich], [rich], [rich], [rich], [rich...
3          2            [[check], [trust], [financial], [bank]]
4          3                                           [[fund]]
5          4                                       [[material]]
6          5                        [[poor], [famous], [black]]
7          6  [[cash], [cash], [cash], [cash], [cash], [cash...
8         12                            [[smell], [hay], [old]]
9         13      [[known], [future], [president], [pr

In [120]:
for item in df['item'].unique():
    tmp = df.loc[df['item']==item,['originality_'+sp for sp in splist]]
    tmp=(tmp-tmp.mean())/tmp.std()
    for sp in splist:
        df.loc[df['item']==item,'orig_'+sp]=tmp['originality_'+sp]

In [121]:
np.round(df[['id','item','response']+['min_'+sp for sp in splist]+['orig_'+sp for sp in splist]],3).head()

,id,item,response,min_cbow_6_ukwac_subtitle,min_cbow_subtitle,min_glove_6B,min_TASA,orig_cbow_6_ukwac_subtitle,orig_cbow_subtitle,orig_glove_6B,orig_TASA
0,3,bureau,federal,0.636,0.624,0.590,0.746,-1.456,-0.319,-1.559,-0.825
1,3,bureau,wardrobe,0.895,0.919,1.049,0.994,-0.007,0.308,0.624,-0.099
2,3,bureau,law,0.825,0.913,0.705,0.999,-1.456,0.936,-1.559,-0.825
3,3,bureau,enforcement,0.692,0.623,0.586,0.736,-1.456,-1.574,-1.013,-0.825
4,3,chamber,cauldron,0.793,0.713,0.918,1.023,1.079,0.329,1.154,0.272


In [122]:
yy_metrics=['ewm_'+sp for sp in splist]+['min_'+sp for sp in splist]+['orig_'+sp for sp in splist]

np.round(df[['id','item','response']+yy_metrics],3).to_csv(sourceFolder+studytag+'_yy_long.csv',index=False)

In [123]:
np.round(df[yy_metrics].corr(),2)

,ewm_cbow_6_ukwac_subtitle,ewm_cbow_subtitle,ewm_glove_6B,ewm_TASA,min_cbow_6_ukwac_subtitle,min_cbow_subtitle,min_glove_6B,min_TASA,orig_cbow_6_ukwac_subtitle,orig_cbow_subtitle,orig_glove_6B,orig_TASA
ewm_cbow_6_ukwac_subtitle,1.00,0.78,0.76,0.54,0.99,0.78,0.74,0.54,0.38,0.33,0.42,0.43
ewm_cbow_subtitle,0.78,1.00,0.64,0.51,0.78,1.00,0.64,0.51,0.23,0.28,0.31,0.36
ewm_glove_6B,0.76,0.64,1.00,0.48,0.74,0.64,0.98,0.48,0.33,0.26,0.39,0.38
ewm_TASA,0.54,0.51,0.48,1.00,0.54,0.51,0.48,1.00,0.16,0.18,0.23,0.25
min_cbow_6_ukwac_subtitle,0.99,0.78,0.74,0.54,1.00,0.78,0.75,0.54,0.37,0.33,0.42,0.43
min_cbow_subtitle,0.78,1.00,0.64,0.51,0.78,1.00,0.64,0.51,0.23,0.28,0.31,0.36
min_glove_6B,0.74,0.64,0.98,0.48,0.75,0.64,1.00,0.48,0.32,0.25,0.39,0.39
min_TASA,0.54,0.51,0.48,1.00,0.54,0.51,0.48,1.00,0.16,0.18,0.23,0.25
orig_cbow_6_ukwac_subtitle,0.38,0.23,0.33,0.16,0.37,0.23,0.32,0.16,1.00,0.52,0.61,0.49
orig_cbow_subtitle,0.33,0.28,0.26,0.18,0.33,0.28,0.25,0.18,0.52,1.00,0.66,0.54


## dispersion

In [44]:

alldata=[]
for i,g in df.groupby(['id','item']):
    tmp=[i[0],i[1]]
    for sp in splist:
        tmp=tmp+list(utilities.dispersion_vectors(g['response_processed'].values.tolist(),semspace_dict[sp]))
    alldata.append(tmp)
columnnames=['id','item']
for sp in splist:
    columnnames=columnnames+['dispMax_'+sp,'dispVar_'+sp]
disp_df=pd.DataFrame(alldata,columns=columnnames)
disp_df.head()

cannot find moola
cannot find moola
cannot find decapitate
cannot find moola
cannot find moola
cannot find basilisk
cannot find jedi
cannot find welleducated
cannot find welleducated
cannot find welleducated
cannot find welleducated
cannot find rulebook
cannot find crat
cannot find crat
cannot find crat
cannot find crat
cannot find sixed
cannot find dooty
cannot find dooty
cannot find dooty
cannot find guesser
cannot find headwear
cannot find moola
cannot find moola
cannot find dinero
cannot find snood
cannot find snood
cannot find amice
cannot find headscarf
cannot find amice
cannot find moolah
cannot find greenies
cannot find purchasable
cannot find purchasable
cannot find hoodie
cannot find hoodie
cannot find ebonics
cannot find paralegal
cannot find catacomb
cannot find smackaroos
cannot find smackaroos
cannot find smackaroos
cannot find thanatology
cannot find thanatology
cannot find thanatology
cannot find moolah
cannot find boyz
cannot find moolah
cannot find moolah
cannot find 

,id,item,dispMax_cbow_6_ukwac_subtitle,dispVar_cbow_6_ukwac_subtitle,dispMax_cbow_subtitle,dispVar_cbow_subtitle,dispMax_glove_6B,dispVar_glove_6B,dispMax_TASA,dispVar_TASA
0,3,bureau,0.584638,0.375036,0.595294,0.394507,0.697951,0.396985,0.621038,0.433555
1,3,chamber,0.407115,0.340872,0.531073,0.426674,0.469552,0.411726,0.488376,0.417068
2,3,death,0.262914,0.227947,0.393291,0.331499,0.325106,0.263138,0.397079,0.349913
3,3,duty,0.470556,0.377731,0.491855,0.450902,0.408750,0.325318,0.515792,0.513402
4,3,expert,0.367348,0.314526,0.393040,0.366057,0.365987,0.287648,0.437042,0.394126


In [45]:
disp_df.to_csv(sourceFolder+'s5_header.csv',index=False)

In [26]:
alldata=[]
for i,g in df.groupby(['id','item']):
    tmp=[i[0],i[1]]
    for sp in splist:
        response_vector=[utilities.get_ew_multiplied_vector(x, semspace_dict[sp]) for x in g['response_processed'].values]
        response_vector = [x for x in response_vector if type(x)!=float]
        y_vec=response_vector-response_vector[0]
        y_vec=y_vec[1:]
        space_rank=len(y_vec)
        if len(y_vec)<2:
            tmp=tmp+[np.nan]
        else:
            try:
                u, s, vh = np.linalg.svd(np.asarray(y_vec).T, full_matrices=True)
                reduced_vect = (np.dot(np.asarray(y_vec),u[:,:space_rank]))
                tmp=tmp+[utilities.getMinVolEllipse(reduced_vect.T)]
            except:
                tmp = tmp+[np.nan]
    alldata.append(tmp)
 

cannot find pully
cannot find lockedyou
cannot find theyou
cannot find openit
cannot find pully
cannot find lockedyou
cannot find theyou
cannot find openit
cannot find lockedyou
cannot find theyou
cannot find openit
cannot find pully
cannot find lockedyou
cannot find keypad
cannot find theyou
cannot find openit
cannot find 4
cannot find illusional
cannot find 4
cannot find illusional
cannot find illusional
cannot find illusional
cannot find brickman
cannot find theyre
cannot find repellant
cannot find lifesize
cannot find lifesize
cannot find bookend
cannot find shard
cannot find iphone
cannot find exfoliant
cannot find exfoliant
cannot find exfoliant
cannot find exfoliant
cannot find withlike
cannot find withlike
cannot find withlike
cannot find withlike
cannot find firepit
cannot find firepit
cannot find flattener
cannot find flattener
cannot find flattener
cannot find flattener
cannot find bookbags
cannot find oldowan
cannot find brickball
cannot find brickly
cannot find bookbags
ca

In [28]:

volume_df=pd.DataFrame(alldata,columns=['id','item']+['volume_'+sp for sp in splist])
tmp=volume_df[['volume_'+sp for sp in splist]]
volume_df[['volume_'+sp for sp in splist]]=(tmp-tmp.mean())/tmp.std()

In [29]:
volume_df.describe()

,id,volume_cbow_6_ukwac_subtitle,volume_cbow_subtitle,volume_glove_6B,volume_TASA
count,133.000000,1.280000e+02,1.270000e+02,1.270000e+02,1.250000e+02
mean,67.646617,3.582204e-16,9.179009e-17,2.159253e-16,-1.998401e-17
std,38.936077,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,1.000000,-6.691108e-01,-6.200338e-01,-1.281694e+00,-3.258897e-01
25%,34.000000,-5.781054e-01,-5.406291e-01,-7.052315e-01,-3.233242e-01
50%,68.000000,-3.313711e-01,-3.197614e-01,-2.002781e-01,-2.931169e-01
75%,101.000000,7.543812e-02,7.381050e-02,5.212745e-01,-1.979177e-01
max,134.000000,5.278853e+00,5.594113e+00,3.970603e+00,6.124143e+00


In [219]:
disp_df.head()

,id,item,dispMax_cbow_6_ukwac_subtitle,dispVar_cbow_6_ukwac_subtitle,dispMax_cbow_subtitle,dispVar_cbow_subtitle,dispMax_glove_6B,dispVar_glove_6B,dispMax_TASA,dispVar_TASA,volume_cbow_6_ukwac_subtitle,volume_cbow_subtitle,volume_glove_6B,volume_TASA
0,2001,box,0.749283,0.580807,0.647501,0.591670,0.781526,0.569401,0.682582,0.581434,-0.674848,-0.677255,-0.701427,-0.499980
1,2001,rope,0.797675,0.576472,0.636408,0.560133,0.699839,0.475446,0.653148,0.587352,-0.671026,-0.647638,-0.797520,-0.557225
2,2002,box,0.844497,0.693563,0.859261,0.723186,1.005941,0.714656,0.887932,0.746202,0.755808,0.595789,1.651976,-0.082294
3,2002,rope,0.866859,0.647170,0.837985,0.642647,0.873515,0.676246,0.894966,0.736614,2.476044,2.105316,2.343181,0.711894
4,2003,box,0.848015,0.596591,0.905790,0.658570,1.239153,0.693978,0.846206,0.760578,4.234262,4.460584,3.625510,3.963930


In [36]:
disp_df= disp_df.merge(volume_df)
disp_df.head()

,id,item,dispMax_cbow_6_ukwac_subtitle,dispVar_cbow_6_ukwac_subtitle,dispMax_cbow_subtitle,dispVar_cbow_subtitle,dispMax_glove_6B,dispVar_glove_6B,dispMax_TASA,dispVar_TASA,volume_cbow_6_ukwac_subtitle,volume_cbow_subtitle,volume_glove_6B,volume_TASA
0,1,brick,0.615696,0.447519,0.713857,0.557964,0.667059,0.491919,0.820077,0.661976,2.179015,2.995090,0.054391,5.136855
1,2,brick,0.542727,0.482022,0.567438,0.492465,1.223400,0.734513,0.545761,0.496554,-0.667892,-0.618995,-1.229259,-0.325814
2,3,brick,0.932081,0.682974,0.760269,0.682665,1.680928,0.848964,0.732203,0.676481,-0.641244,-0.591787,-0.789294,-0.325751
3,4,brick,1.012332,0.703172,0.889717,0.669488,1.918976,0.986956,0.905811,0.734188,-0.664477,-0.614212,-0.752028,-0.325880
4,5,brick,0.700874,0.548392,0.635811,0.564322,1.395373,0.669017,0.660273,0.593285,-0.611188,-0.563631,-1.179481,-0.320687


In [33]:
np.round(disp_df.corr().loc[['dispVar_'+sp for sp in splist],['volume_'+sp for sp in splist]],3)

,volume_cbow_6_ukwac_subtitle,volume_cbow_subtitle,volume_glove_6B,volume_TASA
dispVar_cbow_6_ukwac_subtitle,-0.004,-0.067,0.368,-0.257
dispVar_cbow_subtitle,0.116,0.066,0.490,-0.112
dispVar_glove_6B,-0.168,-0.192,0.094,-0.268
dispVar_TASA,0.332,0.277,0.661,0.093


In [16]:
disp_df.merge(df.groupby(['id','item']).count()[['response']].rename(columns={'response':'fluency'}).reset_index(),left_on=('id','item'),right_on=('id','item')).corr()

,id,dispMax_cbow_6_ukwac_subtitle,dispVar_cbow_6_ukwac_subtitle,dispMax_cbow_subtitle,dispVar_cbow_subtitle,dispMax_glove_6B,dispVar_glove_6B,dispMax_TASA,dispVar_TASA,volume_cbow_6_ukwac_subtitle,volume_cbow_subtitle,volume_glove_6B,volume_TASA,fluency
id,1.000000,0.007669,-0.033920,-0.029532,-0.037351,-0.123933,-0.107611,-0.036177,-0.030573,-0.038615,-0.055789,-0.079730,-0.025800,-0.089609
dispMax_cbow_6_ukwac_subtitle,0.007669,1.000000,0.893296,0.910115,0.891922,0.529979,0.585847,0.899517,0.900847,0.522601,0.504058,0.638963,0.326391,0.724570
dispVar_cbow_6_ukwac_subtitle,-0.033920,0.893296,1.000000,0.882090,0.958255,0.581993,0.698867,0.880168,0.925483,0.321285,0.282783,0.501625,0.069888,0.620409
dispMax_cbow_subtitle,-0.029532,0.910115,0.882090,1.000000,0.939540,0.513276,0.607147,0.944499,0.951407,0.694513,0.656182,0.790000,0.441986,0.799567
dispVar_cbow_subtitle,-0.037351,0.891922,0.958255,0.939540,1.000000,0.561251,0.669960,0.925995,0.966005,0.484769,0.452219,0.653800,0.248519,0.702619
dispMax_glove_6B,-0.123933,0.529979,0.581993,0.513276,0.561251,1.000000,0.865321,0.498995,0.529145,0.033264,0.036828,0.142159,0.021578,0.323406
dispVar_glove_6B,-0.107611,0.585847,0.698867,0.607147,0.669960,0.865321,1.000000,0.596831,0.641526,0.134500,0.126489,0.245837,0.016400,0.391222
dispMax_TASA,-0.036177,0.899517,0.880168,0.944499,0.925995,0.498995,0.596831,1.000000,0.971257,0.651218,0.633509,0.764124,0.441054,0.787107
dispVar_TASA,-0.030573,0.900847,0.925483,0.951407,0.966005,0.529145,0.641526,0.971257,1.000000,0.656508,0.630227,0.777830,0.419681,0.766033
volume_cbow_6_ukwac_subtitle,-0.038615,0.522601,0.321285,0.694513,0.484769,0.033264,0.134500,0.651218,0.656508,1.000000,0.947985,0.924644,0.737857,0.833643


In [21]:

b_tmp = disp_df[disp_df['item']=='box'].rename(columns=dict([('dispVar_'+sp,'b_disp_'+sp) for sp in splist]))[['b_disp_'+sp for sp in splist]+['id']]
r_tmp = disp_df[disp_df['item']=='rope'].rename(columns=dict([('dispVar_'+sp,'r_disp_'+sp) for sp in splist]))[['r_disp_'+sp for sp in splist]+['id']]


In [17]:
#s1 experiment
b_tmp = disp_df[disp_df['item']=='box'].rename(columns=dict([('dispVar_'+sp,'b_disp_'+sp) for sp in splist]+[('volume_'+sp,'b_volume_'+sp) for sp in splist]))[['b_disp_'+sp for sp in splist]+['b_volume_'+sp for sp in splist]+['id']]
r_tmp = disp_df[disp_df['item']=='rope'].rename(columns=dict([('dispVar_'+sp,'r_disp_'+sp) for sp in splist]+[('volume_'+sp,'r_volume_'+sp) for sp in splist]))[['r_disp_'+sp for sp in splist]+['r_volume_'+sp for sp in splist]+['id']]


In [18]:
b_tmp['b_fluency']=df[df['item']=='box'].groupby(['id']).count()['response'].values
r_tmp['r_fluency']=df[df['item']=='rope'].groupby(['id']).count()['response'].values

In [19]:
merged=b_tmp.merge(r_tmp,how='outer',left_on='id',right_on='id')

In [37]:
#for s3
merged=disp_df[disp_df['item']=='brick'].rename(columns=dict([('dispVar_'+sp,'br_disp_'+sp) for sp in splist]+[('volume_'+sp,'br_volume_'+sp) for sp in splist]))[['br_disp_'+sp for sp in splist]+['br_volume_'+sp for sp in splist]+['id']]
merged['br_fluency']=df[df['item']=='brick'].groupby(['id']).count()['response'].values

In [126]:
#for s5
merged=disp_df.rename(columns=dict([('dispVar_'+sp,'disp_'+sp) for sp in splist]))[['disp_'+sp for sp in splist]+['id']]
tmp=df.groupby((['id'])).mean()[yy_metrics]

In [185]:
#for s5 experiment
merged=disp_df.rename(columns=dict([('dispVar_'+sp,'disp_'+sp) for sp in splist]))[['disp_'+sp for sp in splist]+['volume_'+sp for sp in splist]+['id']]
tmp=df.groupby((['id'])).mean()[yy_metrics]

In [215]:
tmp=df.groupby((['id','item'])).mean()[yy_metrics]

KeyError: "None of [Index(['ewm_cbow_6_ukwac_subtitle', 'ewm_cbow_subtitle', 'ewm_glove_6B',\n       'ewm_TASA', 'min_cbow_6_ukwac_subtitle', 'min_cbow_subtitle',\n       'min_glove_6B', 'min_TASA', 'orig_cbow_6_ukwac_subtitle',\n       'orig_cbow_subtitle', 'orig_glove_6B', 'orig_TASA'],\n      dtype='object')] are in the [columns]"

In [27]:
idx = pd.IndexSlice
b_tmp=tmp.loc[idx[:,'box'],:].rename(columns=dict([(m,'b_'+m) for m in yy_metrics]))
r_tmp=tmp.loc[idx[:,'rope'],:].rename(columns=dict([(m,'r_'+m) for m in yy_metrics]))

In [28]:
merged = merged.merge(b_tmp.reset_index().drop(['item'],axis=1),how='outer',left_on='id',right_on='id').merge(
    r_tmp.reset_index().drop(['item'],axis=1),how='outer',left_on='id',right_on='id')

In [97]:
#for s3
merged = merged.merge(tmp.reset_index().drop(['item'],axis=1).rename(columns=dict([(m,'br_'+m) for m in yy_metrics])))

In [186]:
#for s5
merged = merged.merge(tmp.reset_index())

In [187]:
merged.describe()

,disp_cbow_6_ukwac_subtitle,disp_cbow_subtitle,disp_glove_6B,disp_TASA,volume_cbow_6_ukwac_subtitle,volume_cbow_subtitle,volume_glove_6B,volume_TASA,id,ewm_cbow_6_ukwac_subtitle,...,ewm_glove_6B,ewm_TASA,min_cbow_6_ukwac_subtitle,min_cbow_subtitle,min_glove_6B,min_TASA,orig_cbow_6_ukwac_subtitle,orig_cbow_subtitle,orig_glove_6B,orig_TASA
count,464.000000,464.000000,464.000000,464.000000,452.000000,458.000000,455.000000,443.000000,464.000000,464.000000,...,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000
mean,0.362392,0.396970,0.369707,0.451921,0.006819,0.005863,0.018287,0.000674,75.293103,0.744683,...,0.754990,0.899503,0.743454,0.820388,0.751808,0.899782,0.001079,0.001216,0.001354,0.000848
std,0.048929,0.045256,0.066545,0.057509,0.007680,0.003905,0.025262,0.002938,47.824737,0.038130,...,0.051618,0.034759,0.037620,0.038802,0.049729,0.034660,0.204323,0.255272,0.255185,0.257602
min,0.000000,0.207600,0.187100,0.000000,0.000086,0.000080,0.000020,0.000001,3.000000,0.674409,...,0.660025,0.836740,0.674409,0.752775,0.660025,0.836740,-0.501614,-0.487736,-0.480358,-0.426494
25%,0.334246,0.368367,0.325284,0.431931,0.005041,0.004484,0.012658,0.000220,34.000000,0.717710,...,0.724230,0.874611,0.717710,0.792942,0.719489,0.874611,-0.168502,-0.158196,-0.141997,-0.205977
50%,0.363927,0.402940,0.368076,0.465498,0.006235,0.005876,0.016180,0.000442,73.500000,0.738643,...,0.746935,0.892162,0.734218,0.811736,0.745097,0.894333,-0.024326,-0.063236,-0.034999,-0.025848
75%,0.394044,0.429917,0.411155,0.487560,0.007468,0.006894,0.018809,0.000745,118.000000,0.762963,...,0.779963,0.921249,0.762213,0.839872,0.776766,0.921249,0.166378,0.175930,0.142477,0.161830
max,0.484367,0.506920,0.620950,0.529375,0.118078,0.065658,0.369793,0.061505,164.000000,0.845817,...,0.900182,0.991649,0.845817,0.929328,0.900182,0.991649,0.556102,0.693340,0.631766,0.661693


In [130]:
np.round(merged[['id']+[c for c in merged.columns if c!='id']],3).to_csv(sourceFolder+studytag+'_yy_agg.csv',index=False)

In [188]:
#s5 experiment
np.round(merged[['id']+[c for c in merged.columns if c!='id']],3).to_csv(sourceFolder+studytag+'_yy_agg_exp.csv',index=False)

In [38]:
#s1 experiment
np.round(merged[['id']+[c for c in merged.columns if c!='id']],3).to_csv(sourceFolder+'s3_yy_agg_exp.csv',index=False)

# CRJ study

In [50]:
goodItems=['belt','brick','broom','bucket','candle','clock','comb','knife','lamp','pencil','pillow','purse','sock']

In [52]:
studytag='study2'
#studytag='study2'
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
outputfolder='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/algos/'
file ='CRJ_'+studytag+'.csv'
#df=pd.read_csv(sourceFolder+file,sep=';')
df=pd.read_csv(sourceFolder+file)
df.head()

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,SemDis_cbowsubtitle_nf_m,SemDis_cbowBNCwikiukwac_nf_m,SemDis_TASA_nf_m,SemDis_glove_nf_m,SemDis_MEAN,SemDis_factor
0,84176,ruler,ruler,pencil,pencil,0.74268,0.98194,0.82905,0.85553,0.97279,0.87640,-0.05087
1,84176,nose picker,nose picker,pencil,pencil,0.98844,0.99802,0.97247,0.85275,0.98326,0.95899,0.08323
2,84176,scratch ear,scratch ear,pencil,pencil,1.01106,1.02933,1.00287,0.96711,1.10945,1.02396,0.13476
3,84176,hammer,hammer,clock,clock,0.81187,0.86602,0.85598,0.98318,0.83996,0.87140,-0.05508
4,84176,table,table,clock,clock,0.76489,0.80443,0.82374,1.04495,0.74535,0.83667,-0.10461


In [53]:
print(len(df))
df=df[df['item'].isin(goodItems)]
print(len(df),len(df['id'].unique()))

      

6057
4652 149


In [54]:

#lemmatize
df['response_processed']=df.apply(lambda r: utilities.lemmatize(r['response'],utilities.stopwords_edited), axis=1)
#remove stim word
df['response_processed']=df.apply(lambda r: [x for x in r['response_processed'] if x!=r['item']], axis=1)


In [55]:
for sp in splist:
    df['wad'+sp]=df.apply(lambda r: utilities.get_weightedscore(r['item'], r['response_processed'],semspace_dict[sp]), axis=1)
    df['ewm_'+sp]=df.apply(lambda r: utilities.get_ewm_cosdist(r['item'], r['response_processed'], semspace_dict[sp]), axis=1)
    df['mad_'+sp]=df.apply(lambda r: utilities.get_minvec(r['item'], r['response_processed'], semspace_dict[sp]), axis=1)
  

cannot find toliet
cannot find toliet
cannot find cavitity
cannot find odifier
cannot find deodifier
cannot find quittich
cannot find devicfe
cannot find hortibly
cannot find cluttery
cannot find brickfight
cannot find loofa
cannot find sticj
cannot find papperweight
cannot find eyelines
cannot find prentend
cannot find assessory
cannot find tableweight
cannot find assessory
cannot find coater
cannot find straching
cannot find bacl
cannot find pranking
cannot find hairtie
cannot find backscratcher
cannot find mannequinn
cannot find pegleg
cannot find earrig
cannot find snowglobe
cannot find 3
cannot find dogh
cannot find poisioning
cannot find gutiar
cannot find resdy
cannot find haloween
cannot find kmask
cannot find theift
cannot find lanturn
cannot find hannukah
cannot find lipgloss
cannot find stepstool
cannot find tournaquet
cannot find scupltures
cannot find maniquin
cannot find detal
cannot find restrainment
cannot find selfie
cannot find stabbling
cannot find buring
cannot find

cannot find eyelines
cannot find prentend
cannot find assessory
cannot find tableweight
cannot find assessory
cannot find coater
cannot find straching
cannot find bacl
cannot find pranking
cannot find hairtie
cannot find backscratcher
cannot find mannequinn
cannot find pegleg
cannot find earrig
cannot find snowglobe
cannot find 3
cannot find dogh
cannot find poisioning
cannot find gutiar
cannot find resdy
cannot find haloween
cannot find kmask
cannot find theift
cannot find lanturn
cannot find hannukah
cannot find lipgloss
cannot find stepstool
cannot find tournaquet
cannot find scupltures
cannot find maniquin
cannot find detal
cannot find restrainment
cannot find selfie
cannot find stabbling
cannot find buring
cannot find jumprope
cannot find covneyer
cannot find insode
cannot find builda
cannot find furniature
cannot find hairtie
cannot find carseat
cannot find somethibg
cannot find manequin
cannot find cirlce
cannot find scrather
cannot find zipline
cannot find slce
cannot find stab

cannot find theift
cannot find lanturn
cannot find unsharp
cannot find lipgloss
cannot find stepstool
cannot find tournaquet
cannot find scupltures
cannot find maniquin
cannot find detal
cannot find restrainment
cannot find selfie
cannot find stabbling
cannot find buring
cannot find jumprope
cannot find mesure
cannot find covneyer
cannot find insode
cannot find builda
cannot find furniature
cannot find facemask
cannot find hairtie
cannot find carseat
cannot find somethibg
cannot find manequin
cannot find cirlce
cannot find scrather
cannot find slce
cannot find stabing
cannot find poweroutage
cannot find bathroo
cannot find boyfirends
cannot find cousion
cannot find completer
cannot find apnts
cannot find hygge
cannot find homeworl
cannot find comg
cannot find comg
cannot find turket
cannot find cleanii
cannot find didilo
cannot find earmuff
cannot find pplsdtiv
cannot find colar
cannot find colar
cannot find popcial
cannot find seld
cannot find wepon
cannot find wepon
cannot find wepon

cannot find didilo
cannot find earmuff
cannot find pplsdtiv
cannot find colar
cannot find colar
cannot find peice
cannot find popcial
cannot find seld
cannot find wepon
cannot find wepon
cannot find wepon
cannot find doorstopper
cannot find grai
cannot find cutticle
cannot find bunjee
cannot find pencilcase
cannot find mirrior
cannot find doorstopper
cannot find jewerly
cannot find drsw
cannot find killimg
cannot find hairtie
cannot find stepstool
cannot find jeams
cannot find marshmellow
cannot find acessory
cannot find heatless
cannot find scrater
cannot find scracher
cannot find jewlery
cannot find lightshows
cannot find heaadphones
cannot find houe
cannot find unbotton
cannot find emcompassing
cannot find turnaquetting
cannot find wittle
cannot find wieght
cannot find striaghtedge
cannot find attatch
cannot find receving
cannot find sincher
cannot find blanker
cannot find somoene
cannot find turnakit
cannot find cressing
cannot find cressing
cannot find pictrure
cannot find masterb

cannot find attatch
cannot find receving
cannot find sincher
cannot find blanker
cannot find somoene
cannot find turnakit
cannot find cressing
cannot find cressing
cannot find pictrure
cannot find masterbate
cannot find cossplay
cannot find boogieman
cannot find asmr
cannot find scibble
cannot find hiperventilation
cannot find haibachi
cannot find woofd
cannot find shielf
cannot find vegtables
cannot find chnage
cannot find sheild
cannot find frizbee
cannot find tinme
cannot find suffication
cannot find frizbee
cannot find frizbee
cannot find backscratcher
cannot find atteniton
cannot find positivr
cannot find compeitiong
cannot find dialate
cannot find lampshading
cannot find yourseld
cannot find alaem
cannot find arouond
cannot find freshner
cannot find clithe
cannot find valuter
cannot find logcabin
cannot find vaulables
cannot find freshner
cannot find sharpner
cannot find coupns
cannot find acupunture
cannot find vandilazation
cannot find somethig
cannot find earmuff
cannot find c

cannot find buring
cannot find wip
cannot find jumprope
cannot find mesure
cannot find covneyer
cannot find insode
cannot find builda
cannot find furniature
cannot find facemask
cannot find hairtie
cannot find carseat
cannot find somethibg
cannot find manequin
cannot find cirlce
cannot find scrather
cannot find zipline
cannot find slce
cannot find stabing
cannot find outage
cannot find poweroutage
cannot find bathroo
cannot find throgh
cannot find boyfirends
cannot find puncher
cannot find cousion
cannot find chapstick
cannot find apnts
cannot find hygge
cannot find homeworl
cannot find comg
cannot find comg
cannot find turket
cannot find cleanii
cannot find didilo
cannot find pplsdtiv
cannot find organiser
cannot find wair
cannot find colar
cannot find colar
cannot find leveler
cannot find popcial
cannot find seld
cannot find wepon
cannot find wepon
cannot find wepon
cannot find glovers
cannot find curler
cannot find doorstopper
cannot find grai
cannot find cutticle
cannot find bunjee

cannot find dialate
cannot find lampshading
cannot find yourseld
cannot find alaem
cannot find arouond
cannot find freshner
cannot find candleholder
cannot find clithe
cannot find valuter
cannot find logcabin
cannot find tofu
cannot find airbag
cannot find vaulables
cannot find freshner
cannot find sharpner
cannot find coupns
cannot find acupunture
cannot find chopstick
cannot find vandilazation
cannot find zipline
cannot find chopstick
cannot find zipline
cannot find somethig
cannot find chiar
cannot find massager
cannot find curler
cannot find hairstlye
cannot find desinger
cannot find dooor
cannot find curler
cannot find curler
cannot find laptop
cannot find tshirt
cannot find castel
cannot find wifi
cannot find backscratcher
cannot find bookend
cannot find backscratcher
cannot find boomeranh
cannot find brigde
cannot find comfy
cannot find infront
cannot find pencilcase
cannot find harrypotter
cannot find pillowfight
cannot find weaon
cannot find mittons
cannot find cimney
cannot f

cannot find trying
cannot find filet
cannot find knocking
cannot find somethig
cannot find earmuff
cannot find chiar
cannot find scratcher
cannot find stabber
cannot find hairstlye
cannot find summoning
cannot find putting
cannot find bailing
cannot find desinger
cannot find dooor
cannot find smothering
cannot find straightener
cannot find someting
cannot find pictur
cannot find swatter
cannot find brushing
cannot find detangling
cannot find backscratcher
cannot find bookend
cannot find backscratcher
cannot find boomeranh
cannot find mustache
cannot find brigde
cannot find mustache
cannot find pencilcase
cannot find harrypotter
cannot find pillowfight
cannot find weaon
cannot find mittons
cannot find smelling
cannot find cimney
cannot find whittling
cannot find practive
cannot find transporting
cannot find scooping
cannot find dripping
cannot find proping
cannot find elevating
cannot find coathanger
cannot find sextoy
cannot find booste
cannot find weapom
cannot find putting
cannot fin

cannot find scratcher
cannot find zipline
cannot find trying
cannot find filet
cannot find knocking
cannot find somethig
cannot find earmuff
cannot find chiar
cannot find scratcher
cannot find stabber
cannot find hairstlye
cannot find summoning
cannot find putting
cannot find bailing
cannot find desinger
cannot find dooor
cannot find smothering
cannot find straightener
cannot find someting
cannot find pictur
cannot find swatter
cannot find brushing
cannot find detangling
cannot find backscratcher
cannot find bookend
cannot find backscratcher
cannot find boomeranh
cannot find mustache
cannot find brigde
cannot find mustache
cannot find pencilcase
cannot find harrypotter
cannot find pillowfight
cannot find weaon
cannot find mittons
cannot find smelling
cannot find cimney
cannot find whittling
cannot find practive
cannot find transporting
cannot find scooping
cannot find dripping
cannot find proping
cannot find elevating
cannot find coathanger
cannot find sextoy
cannot find booste
cannot 

cannot find scratcher
cannot find zipline
cannot find trying
cannot find filet
cannot find knocking
cannot find somethig
cannot find earmuff
cannot find chiar
cannot find scratcher
cannot find stabber
cannot find hairstlye
cannot find summoning
cannot find putting
cannot find bailing
cannot find desinger
cannot find dooor
cannot find smothering
cannot find straightener
cannot find someting
cannot find pictur
cannot find swatter
cannot find brushing
cannot find detangling
cannot find backscratcher
cannot find bookend
cannot find backscratcher
cannot find boomeranh
cannot find mustache
cannot find brigde
cannot find mustache
cannot find pencilcase
cannot find harrypotter
cannot find pillowfight
cannot find weaon
cannot find mittons
cannot find smelling
cannot find cimney
cannot find whittling
cannot find practive
cannot find transporting
cannot find scooping
cannot find dripping
cannot find proping
cannot find elevating
cannot find coathanger
cannot find sextoy
cannot find booste
cannot 

In [56]:
df.rename(columns=dict([('ewm_'+sp,'ewm_'+utilities.spNameMap[sp]) for sp in splist]+
                         [('mad_'+sp,'mad_'+utilities.spNameMap[sp]) for sp in splist]+
                         [('wad'+sp,'wad_'+utilities.spNameMap[sp]) for sp in splist]),inplace=True)
df.rename(columns={'SemDis_cbowukwacsubtitle_nf_m':'SemDis_cbowu',
                  'SemDis_cbowsubtitle_nf_m':'SemDis_cbows',
                   'SemDis_cbowBNCwikiukwac_nf_m':'SemDis_cboww',
                   'SemDis_TASA_nf_m':'SemDis_tasa',
                   'SemDis_glove_nf_m':'SemDis_glov'
                  },inplace=True)
#df['rater']=df[['rater1','rater2','rater3','rater4']].mean(axis=1)

In [57]:
df.columns

Index(['id', 'response', 'response_nofill', 'item', 'item_nofill',
       'SemDis_cbowu', 'SemDis_cbows', 'SemDis_cboww', 'SemDis_tasa',
       'SemDis_glov', 'SemDis_MEAN', 'SemDis_factor', 'response_processed',
       'wad_cbowu', 'ewm_cbowu', 'mad_cbowu', 'wad_cbows', 'ewm_cbows',
       'mad_cbows', 'wad_glov', 'ewm_glov', 'mad_glov', 'wad_tasa', 'ewm_tasa',
       'mad_tasa', 'wad_cboww', 'ewm_cboww', 'mad_cboww'],
      dtype='object')

In [58]:

np.round(df,3).to_csv(outputfolder+'CRJ_'+studytag+'_yy_long.csv')

In [89]:
np.round(df[['id','item','response','response_processed']+['ewm_'+sp for sp in splist]+['min_'+sp for sp in splist]],3).describe()

,id,ewm_cbow_6_ukwac_subtitle,ewm_cbow_subtitle,ewm_glove_6B,ewm_TASA,min_cbow_6_ukwac_subtitle,min_cbow_subtitle,min_glove_6B,min_TASA
count,4652.000000,4531.000000,4536.000000,4540.000000,4501.000000,4531.000000,4536.000000,4540.000000,4501.000000
mean,84725.135641,0.848239,0.897442,0.899767,0.944603,0.787502,0.866325,0.849144,0.944597
std,357.992738,0.152905,0.125480,0.159742,0.129963,0.109131,0.109085,0.122556,0.133041
min,84176.000000,0.313000,0.422000,0.345000,0.180000,0.313000,0.422000,0.345000,0.180000
25%,84430.750000,0.738000,0.821000,0.790000,0.911000,0.726000,0.808000,0.774000,0.908000
50%,84671.000000,0.851000,0.913000,0.906000,0.972000,0.797000,0.883000,0.866000,0.979000
75%,85014.000000,0.974500,0.986000,1.013000,1.020000,0.868000,0.941000,0.937000,1.023000
max,85636.000000,1.185000,1.234000,1.367000,1.273000,1.106000,1.127000,1.249000,1.180000


In [90]:
len(df)

4652

In [132]:

alldata=[]
for i,g in df.groupby(['id','item']):
    tmp=[i[0],i[1]]
    for sp in splist:
        try:
            tmp=tmp+list(utilities.dispersion_vectors(g['response_processed'].values.tolist(),semspace_dict[sp]))
        except:
            tmp=tmp+[np.nan,np.nan]
            
        break
    alldata.append(tmp)
columnnames=['id','item']
for sp in splist:
    columnnames=columnnames+['dispMax_'+sp,'dispVar_'+sp]
disp_df=pd.DataFrame(alldata,columns=columnnames)
disp_df.head()

cannot find waterer
cannot find awhirl
cannot find larping


ValueError: 10 columns passed, passed data had 4 columns

In [134]:
g['response_processed'].values.tolist()

[['dry', 'paint', 'brush']]

In [135]:
utilities.dispersion_vectors(g['response_processed'].values.tolist(),semspace_dict[sp])

(2.220446049250313e-16, 2.220446049250313e-16)

In [95]:
df['ewm']=df[['ewm_'+sp for sp in splist]].mean(axis=1)
df['min']=df[['min_'+sp for sp in splist]].mean(axis=1)
#df['rater']=df[['rater1','rater2','rater3','rater4']].mean(axis=1)
df_agg = df.groupby(['id','item']).max()[['ewm','min']]
df_agg.rename(columns={'ewm':'ewm_Max','min':'min_Max'},inplace=True)
tmp = df.groupby(['id','item']).mean()[['ewm','min']]
df_agg=pd.concat([df_agg,
                  tmp.rename(columns={'ewm':'ewm_Mean','min':'min_Mean'})],axis=1)
df_agg=pd.concat([df_agg,df.groupby(['id','item']).count()[['ewm']].rename(columns={'ewm':'fluency'})],axis=1)
#df_agg=pd.concat([df_agg,df.groupby(['id','item']).mean()[['rater']]],axis=1)
len(df_agg)
                  

1838

In [77]:
df.corr().loc['rater']

id                          -0.062057
rater1                       0.854568
rater2                       0.757332
rater3                       0.804938
rater4                       0.843036
ewm_cbow_6_ukwac_subtitle    0.354987
min_cbow_6_ukwac_subtitle    0.400462
ewm_cbow_subtitle            0.362445
min_cbow_subtitle            0.365777
ewm_glove_6B                 0.309282
min_glove_6B                 0.348371
ewm_TASA                     0.283366
min_TASA                     0.361416
ewm                          0.375196
min                          0.424088
rater                        1.000000
Name: rater, dtype: float64

In [97]:
tmp=disp_df[['dispVar_'+sp for sp in splist]]
disp_df['disp']=((tmp-tmp.mean())/tmp.std()).mean(axis=1)
#df_agg = df_agg.merge(disp_df,how='outer',left_on=['id','item'],right_on=['id','item'])[['id','item','ewm_Max','min_Max','ewm_Mean','min_Mean','disp','fluency','rater']]
df_agg = df_agg.merge(disp_df,how='outer',left_on=['id','item'],right_on=['id','item'])[['id','item','ewm_Max','min_Max','ewm_Mean','min_Mean','disp','fluency']]

In [98]:
np.round(df_agg,3).to_csv(outputfolder+'CRJ_'+studytag+'yy_agg_item.csv')

In [122]:
studytag='study1'
file ='CRJ_'+studytag+'_survey.csv'
agg=pd.read_csv(sourceFolder+file)
agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 8 columns):
subject                   186 non-null int64
d_age                     186 non-null object
d_gender                  186 non-null object
bicb                      186 non-null object
o_bfas                    186 non-null object
i_bfas                    186 non-null object
ssci_creative_identity    186 non-null object
ssci_creative_efficacy    186 non-null object
dtypes: int64(1), object(7)
memory usage: 11.8+ KB


In [123]:
agg = agg.replace(r'^\s*$', np.NaN, regex=True)

In [124]:
aggdf=agg.rename(columns={'subject':'id'}).merge(df_agg.groupby('id').mean().reset_index(),how='outer',left_on='id',right_on='id')
aggdf.head()

,id,d_age,d_gender,bicb,o_bfas,i_bfas,ssci_creative_identity,ssci_creative_efficacy,ewm_Max,min_Max,ewm_Mean,min_Mean,disp,fluency
0,2000,42,1,0.058823529,4.2,3.1,3,3.666666667,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,23,1,0.454545455,3.9,4.3,3.5,4.666666667,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,27,0,0.705882353,3.7,3.8,4.4,4.333333333,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,24,0,0.264705882,4.6,3.8,4.6,4.333333333,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,18,1,0,3.6,3.7,4.4,3.666666667,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
aggdf[['id']+col2].to_csv(outputfolder+'CRJ_'+studytag+'yy_agg.csv')